# `Interpolate`
----

In [ ]:
import numpy as np
from numpy.typing import NDArray
from typing import TypeVar

F = TypeVar("F", float, np.floating)

def _interpolate(image: NDArray[F], x: F, y: F) -> F:
    """
    Performs bilinear interpolation on an image at a specified (x, y) coordinate.

    Parameters:
    -----------
    image : NDArray[F]
        A NumPy array representing the image (height, width, channels).
    x : F
        The x-coordinate (can be a floating-point value).
    y : F
        The y-coordinate (can be a floating-point value).

    Returns:
    --------
    F
        The interpolated pixel value at (x, y).

    Notes:
    ------
    - Uses bilinear interpolation, considering the four nearest pixels.
    - Clamps coordinates to ensure they remain within bounds.
    - Works with multi-channel images (e.g., RGB).
    - Returns a floating-point interpolated pixel value.

    Complexity:
    -----------
    - O(1), since only four neighboring pixels are accessed.
    """

    h, w, c = image.shape  # Image dimensions

    # Ensure coordinates are within bounds
    x = np.clip(x, 0, w - 1)
    y = np.clip(y, 0, h - 1)

    # Get neighboring pixel coordinates
    x0 = int(np.floor(x))
    y0 = int(np.floor(y))
    x1 = min(x0 + 1, w - 1)
    y1 = min(y0 + 1, h - 1)

    # Compute interpolation weights
    xw = x - x0
    yw = y - y0

    # Convert pixel values to float for proper interpolation
    p00 = image[y0, x0].astype(np.float32)  # Top-left
    p01 = image[y0, x1].astype(np.float32)  # Top-right
    p10 = image[y1, x0].astype(np.float32)  # Bottom-left
    p11 = image[y1, x1].astype(np.float32)  # Bottom-right

    # Perform bilinear interpolation
    return (
        p00 * (1 - xw) * (1 - yw) +
        p01 * xw * (1 - yw) +
        p10 * (1 - xw) * yw +
        p11 * xw * yw
    )

if __name__ == "__main__":
    # Create a sample 5x5 RGB image with random pixel values (0-255)
    image = np.random.randint(0, 256, (5, 5, 3), dtype = np.uint8)

    # Interpolate at a fractional coordinate
    interpolated_value = _interpolate(image, x = 2.3, y = 1.7)

    print("Interpolated Pixel Value:", interpolated_value)


# `Affine`
----

In [ ]:
import numpy as np
from numpy.typing import NDArray
from typing import TypeVar
from collections.abc import Sequence

F = TypeVar("F", float, np.floating )

def _affine(a: NDArray[F] | Sequence[F], *, angle: F = 0.0, translate: NDArray[F] | Sequence[F] = ([0.0, 0.0]), scale: F = 1.0, radians: bool | None = None) -> NDArray[F]:
    """
    Applies a 2D affine transformation to a set of points.

    The function scales, rotates, and translates a given set of 2D points using homogeneous 
    coordinates and matrix multiplication.


    Parameters:
    -----------
    a : NDArray[F] | Sequence[F]
        A (3, N) NumPy array or a sequence representing homogeneous coordinates, 
        where each column corresponds to a point.
    angle : F, default=0.0
        Rotation angle. If `radians` is `False` or `None` (heuristic detection enabled), 
        the function assumes degrees and converts if needed.
    translate : NDArray[F] | Sequence[F], default=[0.0, 0.0]
        A (2,) array or sequence specifying translation along x and y dimensions.
    scale : F, default=1.0
        Scaling factor applied uniformly to both x and y dimensions.
    radians : bool | None, default=None
        If `True`, `angle` is interpreted as radians. If `False`, the function assumes degrees.
        If `None`, heuristic detection is used:
        - If `abs(angle) > 6.283`, assumes degrees and converts.
        - If `abs(angle) > 360`, a warning is issued.

    Returns:
    --------
    NDArray[F]
        The transformed points as a (3, N) NumPy array in homogeneous coordinates.

    Notes:
    ------
    - Input points should be in homogeneous coordinates (3, N), where each column is a point.
    - The function applies transformations in the order: Scaling → Rotation → Translation.
    - Heuristic detection for `angle` ensures automatic handling of degrees vs. radians.
    """

    # Convert inputs to NumPy arrays if needed
    if isinstance(a, Sequence):
        a = np.asarray(a, dtype = np.float32)
    translate = np.asarray(translate, dtype = np.float32)

    # Ensure a is a (3, N) column vector
    if a.shape[0] != 3:
        if a.shape == (1, 3):  
            a = a.T  # Reshape row vector (1, 3) to column vector (3, 1)
        else:
            raise ValueError("Input 'a' must be a (3, N) array in homogeneous coordinates.")
        
    # Convert angle from degrees to radians
    if radians is None:  # Only apply heuristic if not explicitly set
        if abs(angle) > 6.283:  
            print("Warning: Angle appears to be in degrees. Converting to radians.")
            angle = np.radians(angle)
        elif abs(angle) > 360:
            print("Warning: Angle is unusually large. Ensure the correct unit is used.")

    # Scaling matrix
    S = np.array([[scale, 0, 0],
                  [0, scale, 0],
                  [0, 0, 1]])

    # Rotation matrix
    R = np.array([[np.cos(angle), -np.sin(angle), 0],
                  [np.sin(angle), np.cos(angle), 0],
                  [0, 0, 1]], dtype = np.float32)

    # Translation matrix
    T = np.array([[1, 0, translate[0]],
                  [0, 1, translate[1]],
                  [0, 0, 1]], dtype = np.float32)

    # Composite transformation matrix
    M = T @ R @ S  # Apply transformations in order: Scale → Rotate → Translate

    # Transform points
    return M @ a


if __name__ == "__main__":

    # Define transformation parameters
    scale = 3
    angle = 30  # Degrees
    translate = [1, 2]  # List input (ArrayLike)

    # Test cases
    test_cases = [
        [[0.1, 0.3], [1.0, 2.0]],  # List of (x, y) points
        ((0.1, 0.3), (1.0, 2.0)),  # Tuple of (x, y) points
        np.array([[0.1, 0.3], [1.0, 2.0]], dtype = np.float32),  # NumPy array (N, 2)
        np.array([[0.1], [0.3], [1.0]], dtype = np.float32)  # Already (3, 1)
    ]

    for i, a in enumerate(test_cases, 1):
        try:
            transformed = _affine(a = a, scale = scale, angle = angle, translate = translate, radians = False)
            print(f"Test {i} Passed:\n{transformed}\n")
        except ValueError as e:
            print(f"Test {i} Failed: {e}\n")

# `Backwards Mapping [unstable]`
----

### `Slow`

In [7]:
import matplotlib.pyplot as plt
import numpy as np


def _backmap(source: np.ndarray, target: np.ndarray, transform: np.ndarray) -> np.ndarray:
    
    # dimension grabbing
    h, w, _ = source.shape
    target_h, target_w, _ = target.shape

    result = np.copy(target)
    
    # invert transformation matrix for backward mapping
    transform_inv = np.linalg.inv(transform)

    for y in range(target_h):
        for x in range(target_w):

            # homogenize
            # aka add one dimension
            xy_target_homogenized = np.array([x, y, 1]) # (x, y) to (x, y, 1)

            # backward mapping
            xy_backward_map = transform_inv @ xy_target_homogenized.T # convert to column vector by using "T"
            xy_backward_map = np.array(xy_backward_map).flatten() # convert to 1d array

            # homogenous divide to get source coordinates
            x_source = xy_backward_map[0] / xy_backward_map[2]
            y_source  = xy_backward_map[1] / xy_backward_map[2]

            # just another classic boundary check
            if 0 <= x_source < w and 0 <= y_source < h: # we can siimply discard anything not in bounds

                # NOTE FOR FUTURE LOGAN
                ''' 
                    We multiplied the inverse of the transformation matrix with a homogenized x and y from the target image.
                    
                    That process is a "map" to find the exact x and y coords from a source.
                    
                        Now that we've found the position of WHERE the source image's x and y are...

                        ...we interpolate them to grab an estimated pixel color from that location.
                    
                    Once we have the pixel, we set the current x and y coordinate of the target to that pixel.
                '''
                result[y, x] = _interpolate(source, x_source, y_source)
    
    return result


from math import sin, cos, pi

filename = "test.png"
im = plt.imread(filename)
canvas = np.zeros_like(im) + np.array([[[ 80/255, 45/255, 127/255 ]]]) #ECU Purple

transform = np.matrix([[cos(45 * pi/180), -sin(45 * pi/180), im.shape[1]/2],
                       [sin(45 * pi/180), cos(45 * pi/180), -im.shape[0]/5],
                       [0, 0, 1]])

result = compose(im, canvas, transform)

plt.imshow(result, vmin = 0)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'test.png'

### `Fast`

In [6]:
def _fast_backmap(source, target, transform, box = None):   
    
    # dimension grabbing
    h, w, _ = source.shape

    result = np.copy(target)
    
    # invert transformation matrix for backward mapping
    transform_inv = np.linalg.inv(transform)

    for y in range(box[0].y, box[1].y + 1):
        for x in range(box[0].x, box[1].x + 1):

            # homogenize
            # aka add one dimension
            xy_target_homogenized = np.array([x, y, 1]) # (x, y) to (x, y, 1)

            # backward mapping
            xy_backward_map = transform_inv @ xy_target_homogenized.T # convert to column vector by using "T"
            xy_backward_map = np.array(xy_backward_map).flatten() # convert to 1d array

            # homogenous divide to get source coordinates
            x_source = xy_backward_map[0] / xy_backward_map[2]
            y_source  = xy_backward_map[1] / xy_backward_map[2]

            # just another classic boundary check
            if 0 <= x_source < w and 0 <= y_source < h: # we can siimply discard anything not in bounds
                result[y, x] = _interpolate(source, x_source, y_source)
    
    return result

# `Warp`
----

### `Slow`

In [10]:
def _warp(source_im ,target_im, s0, s1, s2, s3, t0, t1, t2, t3):
    '''
        NOTE Dear future Logan,

            Remember, the homography matrix (in this context) is just the transformation matrix.

            If you have all 8 points -- 4 from the source, and 4 from the destination -- the homography 
            matrix describes the relationship of change between the two images.

            In this example, we just took the corner coordinates of the source image, and then the 
            four coordinates of the destination to map the original image to the new location.
    '''
    return _backmap(source_im, (result := np.copy(target_im)), _homography(s0, s1, s2, s3, t0, t1, t2, t3))

### `Fast`

In [13]:
def _fast_warp(source_im,target_im,s0,s1,s2,s3,t0,t1,t2,t3):

    start = Point(np.amin([t0.x, t1.x, t2.x, t3.x]),
                  np.amin([t0.y, t1.y, t2.y, t3.y]))
    
    end = Point(np.amax([t0.x, t1.x, t2.x, t3.x]), 
                np.amax([t0.y, t1.y, t2.y, t3.y]))
    
    return _fast_backmap(source_im, (result := np.copy(target_im)), _homography(s0, s1, s2, s3, t0, t1, t2, t3), (start, end))